pyrebase4 (필수)
firebase-admin
python-firebase

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd

In [3]:
import numpy as np
import csv

import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import time
from time import sleep

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver


import schedule
import time
from datetime import datetime

In [4]:
import pyrebase

In [5]:
def browser():

    url ='https://thinkyou.co.kr/contest/sector.asp'

    browser = Chrome(r"C:\Users\user\Downloads\chromedriver_win32\chromedriver.exe")

    delay=3
    browser.implicitly_wait(delay)

    browser.get(url)

    browser.maximize_window()

    body = browser.find_element_by_tag_name('body')

    try :

        browser.find_elements_by_xpath('//*[@id="searchFrm"]/div/dl[1]/dd/p[6]/label/span')[0].click()
        browser.find_elements_by_xpath('//*[@id="searchFrm"]/div/dl[2]/dd/p[1]/label/span')[0].click()
    except:
        pass

    SCROLL_PAUSE_TIME = 0.5
    while True:
        last_height = browser.execute_script('return document.documentElement.scrollHeight')

        for i in range(3):
            body.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
        new_height = browser.execute_script('return document.documentElement.scrollHeight')
        if new_height == last_height:
            break;

    page = browser.page_source
    soup = BeautifulSoup(page, 'lxml')
    return soup

In [6]:
def crawling():
    soup = browser()

    len_day = 20

    links_bef = []
    titles_bef = []
    dday_bef = []
    inst_bef = []
    start_bef = []
    end_bef = []

    links_aft = []
    titles_aft = []
    inst_aft = []

    for i in range(len_day):
        t = soup.select(' .title > a > dl > dt ')[i].text
        fin = soup.select(' .statNew > p ')[i].text


        if fin == '마감':
            base_url = 'https://thinkyou.co.kr'
            titles_aft.append(soup.select(' .title > a > dl > dt ')[i].text)
            inst_aft.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])

            links_aft.append(base_url + soup.select(' .title > a')[i]['href'][2:])
        else:
            stand = soup.select(' .statNew')[i].text.split('D')[1]


            if stand == '-day':
                base_url = 'https://thinkyou.co.kr'
                titles_bef.append(soup.select(' .title > a > dl > dt ')[i].text)
                inst_bef.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])
                dday_bef.append('D' + soup.select(' .statNew')[i].text.split('D')[1])
                links_bef.append(base_url + soup.select(' .title > a')[i]['href'][2:])
                a = i * 2
                start_bef.append(soup.select(' .etc')[a].text[:8])
                end_bef.append(soup.select(' .etc')[a].text[11:])

            else:

                base_url = 'https://thinkyou.co.kr'
                titles_bef.append(soup.select(' .title > a > dl > dt ')[i].text)
                inst_bef.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])
                dday_bef.append('D' + soup.select(' .statNew')[i].text.split('D')[1])
                links_bef.append(base_url + soup.select(' .title > a')[i]['href'][2:])
                a = i * 2
                start_bef.append(soup.select(' .etc')[a].text[:8])
                end_bef.append(soup.select(' .etc')[a].text[11:])

    tabl_data_bef = {'title': titles_bef, 'notice': start_bef, 'deadline': end_bef, 
                     'dday': dday_bef, 'sponsor': inst_bef, 'title2': titles_bef, 'link': links_bef}

    df_bef = pd.DataFrame(tabl_data_bef, columns=['type', 'title', 'notice', 'deadline', 'dday', 'qualification', 'sponsor', 'title2','link'])
    
    df_bef['type'] = '공모전'
    df_bef['qualification'] = '대학(원)생'


    return df_bef

In [ ]:
# 모든 크롤링 데이터 합쳐서 중복 제거 하는 부분

'''
def sum():
    df1 = crawling()
    df2 = browser2() #이런식으로 크롤링 데이터 합쳐나가면 됨
    mid = pd.concat([df1, df2])
    mid_df = mid.reset_index(drop=True)
    mid_df['title'] = mid_df['title'].str.strip()
    fin_df = mid_df.drop_duplicates(['title'], keep='first')
    fin = fin_df.reset_index(drop = True)
    
    return fin
'''

# Config 보기

개인정보인지 공통된 정보인지 몰라 일단 공란으로 둠

- 프로젝트 설정 > 일반 페이지 하단 '내 앱'의 Firebase SDK snippet을 CDN으로 맞춤

- 아래 나온 정보에 따라 코드에 채우면 됨

In [9]:
def tofb():
    df = crawling() # sum()
    postdata = df.to_dict()
    config = {
        "apiKey": "",
        "authDomain": "",
        "databaseURL": "https://activity-crawling-default-rtdb.firebaseio.com/",
    
        "projectId": "",
        "storageBucket": "",
        "messagingSenderId": "",
        "appId": ""}
    firebase = pyrebase.initialize_app(config)
    db = firebase.database()
    n = 0
    l = len(df)
    for i in df['title']:
        db.child(n).update(df.iloc[n].to_dict())
        n = n+1
        print(n)
        if n ==l : break;
    

In [10]:
tofb()

In [ ]:
def job():
    now = datetime.now()
    print(now)
    tofb()
    print("end")


schedule.every().day.at("00:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)